In [4]:
# Riot LoL Minimal Flow — Get Match History via Riot ID + Match-V5
# Uses ONLY:
#  - /riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}
#  - /lol/match/v5/matches/by-puuid/{puuid}/ids
# -------------------------------------------------------------

from datetime import datetime, timezone
import requests
import time
import pandas as pd
import os

# =============================
# CONFIG
# =============================
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <-- replace with your Riot API key
GAME_NAME = "Hide on bush"       # Riot ID name
TAG_LINE = "KR1"          # Riot ID tagline
REGION = "ASIA"           # choose from: AMERICAS, EUROPE, ASIA
QUEUE = 420               # optional queue filter (420 = ranked solo); set to None for all
OUT_DIR = "output_minimal"
os.makedirs(OUT_DIR, exist_ok=True)

SLEEP_BETWEEN_CALLS = 1.2
RETRY_MAX = 5

# =============================
# HELPERS
# =============================
def _headers():
    return {"X-Riot-Token": API_KEY}

def _request_json(url, params=None):
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=30)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_CALLS)
                return r.json()
            elif r.status_code in (429, 503):
                delay = float(r.headers.get("Retry-After", 1.5 * (2 ** attempt)))
                print(f"Rate limited, sleeping {delay:.1f}s...")
                time.sleep(delay)
            elif r.status_code in (401, 403):
                raise RuntimeError(f"{r.status_code} {r.text[:200]} → Check API key validity or region access.")
            elif r.status_code == 404:
                return None
            else:
                print(f"HTTP {r.status_code}: {r.text[:120]}")
                time.sleep(2)
        except requests.RequestException as e:
            print("Network error:", e)
            time.sleep(2)
    raise RuntimeError(f"Request failed after {RETRY_MAX} attempts.")

# =============================
# STEP 1 — Resolve Riot ID → PUUID (Account-V1)
# =============================
def get_puuid_from_riotid(gameName, tagLine, region):
    url = f"https://{region.lower()}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"
    data = _request_json(url)
    if not data or "puuid" not in data:
        raise RuntimeError(f"Failed to resolve Riot ID {gameName}#{tagLine}. Response: {data}")
    puuid = data["puuid"]
    print(f"✅ {gameName}#{tagLine} → PUUID: {puuid}")
    return puuid

# =============================
# STEP 2 — Get Match IDs (Match-V5)
# =============================
def get_all_match_ids(puuid, region, queue=None):
    base_url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    start = 0
    all_ids = []

    while True:
        params = {"start": start, "count": 100}
        if queue is not None:
            params["queue"] = queue
        ids = _request_json(base_url, params=params)
        if not ids:
            break
        all_ids.extend(ids)
        print(f"Fetched {len(ids)} IDs (total {len(all_ids)})")
        start += len(ids)

    print(f"✅ Total matches found: {len(all_ids)}")
    return all_ids

# =============================
# RUN
# =============================
puuid = get_puuid_from_riotid(GAME_NAME, TAG_LINE, REGION)
match_ids = get_all_match_ids(puuid, REGION, queue=None)

# Save to CSV
out_path = os.path.join(OUT_DIR, f"match_ids_{GAME_NAME}_{TAG_LINE}.csv")
pd.DataFrame({"matchId": match_ids}).to_csv(out_path, index=False)
print("Saved match IDs to:", out_path)

# preview
pd.read_csv(out_path).head()


✅ Hide on bush#KR1 → PUUID: nrlGgzDDgaxt5Xw8kHKTKBExb81_VfVWrtwOiKJ3l47WJngAxULpg4SwueYHIvK1606gzmHhnyt60A
Fetched 100 IDs (total 100)
Fetched 100 IDs (total 200)
Fetched 85 IDs (total 285)


KeyboardInterrupt: 

In [15]:
# =============================
# STEP 3 — Fetch Details for First 10 Matches
# =============================
import json, os

def get_match_detail(match_id, region):
    """Fetch full match JSON for a given matchId from Match-V5."""
    url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    data = _request_json(url)
    if not data:
        print(f"⚠️ Skipping {match_id} (no data)")
        return None
    return data

# Limit to first 10 matches for testing
sample_match_ids = match_ids[:10]
print(f"Fetching details for {len(sample_match_ids)} matches...")

details = []
for i, mid in enumerate(sample_match_ids, start=1):
    info = get_match_detail(mid, REGION)
    if info:
        details.append(info)
    print(f"  [{i}/{len(sample_match_ids)}] Done {mid}")
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Pulled {len(details)} match details")

# Save raw JSON (optional)
out_json = os.path.join(OUT_DIR, f"match_details_sample_{GAME_NAME}_{TAG_LINE}.json")
with open(out_json, "w") as f:
    json.dump(details, f)
print("Saved sample match data to:", out_json)


Fetching details for 10 matches...
  [1/10] Done KR_7768456553
  [2/10] Done KR_7768428775
  [3/10] Done KR_7737155171
  [4/10] Done KR_7694218699
  [5/10] Done KR_7669256754
  [6/10] Done KR_7669237789
  [7/10] Done KR_7669222323
  [8/10] Done KR_7669206621
  [9/10] Done KR_7636932338
  [10/10] Done KR_7599078493
✅ Pulled 10 match details
Saved sample match data to: output_minimal/match_details_sample_Faker_KR1.json


In [16]:
# =============================
# STEP 4 — Fetch Match Timelines (Events / Gold / XP)
# =============================
import json, os

def get_match_timeline(match_id, region):
    """Fetch timeline JSON for a given matchId."""
    url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"
    data = _request_json(url)
    if not data:
        print(f"⚠️ Skipping timeline for {match_id}")
        return None
    return data

# Limit to first 3–5 matches for testing (timeline is large)
timeline_match_ids = match_ids[:5]
timelines = []

for i, mid in enumerate(timeline_match_ids, start=1):
    timeline = get_match_timeline(mid, REGION)
    if timeline:
        timelines.append(timeline)
    print(f"  [{i}/{len(timeline_match_ids)}] timeline done for {mid}")
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Pulled {len(timelines)} match timelines")

# Save raw JSON
out_timeline_json = os.path.join(OUT_DIR, f"match_timelines_sample_{GAME_NAME}_{TAG_LINE}.json")
with open(out_timeline_json, "w") as f:
    json.dump(timelines, f)
print("Saved timeline data to:", out_timeline_json)


  [1/5] timeline done for KR_7768456553
  [2/5] timeline done for KR_7768428775
  [3/5] timeline done for KR_7737155171
  [4/5] timeline done for KR_7694218699
  [5/5] timeline done for KR_7669256754
✅ Pulled 5 match timelines
Saved timeline data to: output_minimal/match_timelines_sample_Faker_KR1.json


In [18]:
timeline_rows = []
for tl in timelines:
    match_id = tl["metadata"]["matchId"]
    for frame in tl["info"]["frames"]:
        tstamp = frame["timestamp"] // 60000  # minutes
        for pid, pdata in frame["participantFrames"].items():
            timeline_rows.append({
                "matchId": match_id,
                "minute": tstamp,
                "participantId": pid,
                "totalGold": pdata["totalGold"],
                "xp": pdata["xp"],
                "level": pdata["level"],
            })
timeline_df = pd.DataFrame(timeline_rows)
timeline_df


,matchId,minute,participantId,totalGold,xp,level
0,KR_7768456553,0,1,500,0,1
1,KR_7768456553,0,2,500,0,1
2,KR_7768456553,0,3,500,0,1
3,KR_7768456553,0,4,500,0,1
4,KR_7768456553,0,5,500,0,1
...,...,...,...,...,...,...
1005,KR_7669256754,23,6,17067,28684,18
1006,KR_7669256754,23,7,14813,26515,18
1007,KR_7669256754,23,8,16864,27554,18
1008,KR_7669256754,23,9,18826,30208,18


In [20]:
player_df = timeline_df[timeline_df['participantId'] == "1"]
player_df

,matchId,minute,participantId,totalGold,xp,level
0,KR_7768456553,0,1,500,0,1
10,KR_7768456553,1,1,905,18,1
20,KR_7768456553,2,1,1073,267,1
30,KR_7768456553,3,1,1993,915,3
40,KR_7768456553,4,1,3015,1556,4
...,...,...,...,...,...,...
960,KR_7669256754,20,1,14716,25973,18
970,KR_7669256754,21,1,15121,26615,18
980,KR_7669256754,22,1,15796,28492,18
990,KR_7669256754,23,1,16156,28792,18


In [10]:
# Flatten participant data into a table
participants = []
for match in details:
    info = match["info"]
    for p in info["participants"]:
        participants.append({
            "matchId": info["gameId"],
            "gameDuration": info["gameDuration"],
            "queueId": info["queueId"],
            "championName": p["championName"],
            "kills": p["kills"],
            "deaths": p["deaths"],
            "assists": p["assists"],
            "win": p["win"],
            "summonerName": p["summonerName"],
            "teamId": p["teamId"],
            "role": p["role"],
            "lane": p["lane"]
        })

df = pd.DataFrame(participants)
print("Participants shape:", df.shape)
df


Participants shape: (106, 12)


,matchId,gameDuration,queueId,championName,kills,deaths,assists,win,summonerName,teamId,role,lane
0,5403251060,1690,420,Rumble,4,6,8,True,,100,SOLO,TOP
1,5403251060,1690,420,Ambessa,13,5,9,True,,100,NONE,JUNGLE
2,5403251060,1690,420,Ahri,4,6,11,True,,100,SOLO,MIDDLE
3,5403251060,1690,420,Riven,14,3,10,True,,100,SOLO,BOTTOM
4,5403251060,1690,420,Blitzcrank,2,3,25,True,,100,NONE,JUNGLE
...,...,...,...,...,...,...,...,...,...,...,...,...
101,5402460752,1096,420,Gwen,5,1,3,True,,200,SUPPORT,NONE
102,5402460752,1096,420,Khazix,10,0,1,True,,200,SUPPORT,NONE
103,5402460752,1096,420,Ahri,4,0,7,True,,200,SUPPORT,NONE
104,5402460752,1096,420,Smolder,2,3,3,True,,200,SUPPORT,NONE


In [11]:
df_match = df[df['matchId'] == 5403251060]
df_match

,matchId,gameDuration,queueId,championName,kills,deaths,assists,win,summonerName,teamId,role,lane
0,5403251060,1690,420,Rumble,4,6,8,True,,100,SOLO,TOP
1,5403251060,1690,420,Ambessa,13,5,9,True,,100,NONE,JUNGLE
2,5403251060,1690,420,Ahri,4,6,11,True,,100,SOLO,MIDDLE
3,5403251060,1690,420,Riven,14,3,10,True,,100,SOLO,BOTTOM
4,5403251060,1690,420,Blitzcrank,2,3,25,True,,100,NONE,JUNGLE
5,5403251060,1690,420,Nidalee,4,6,10,False,,200,SOLO,TOP
6,5403251060,1690,420,JarvanIV,11,7,5,False,,200,NONE,JUNGLE
7,5403251060,1690,420,Mel,7,7,2,False,,200,SOLO,MIDDLE
8,5403251060,1690,420,Jhin,0,8,11,False,,200,CARRY,BOTTOM
9,5403251060,1690,420,Soraka,1,9,5,False,,200,SUPPORT,BOTTOM


# champion-v3

In [2]:
# =============================
# STEP 5 — Champion Rotations (Champion-V3)
# =============================
def get_champion_rotations(platform):
    """
    Fetch current free champion rotation list.
    platform = e.g. 'na1', 'euw1', 'kr', 'br1'
    """
    url = f"https://{platform}.api.riotgames.com/lol/platform/v3/champion-rotations"
    data = _request_json(url)
    if not data:
        print(f"⚠️ No data returned for champion rotations on {platform}")
        return None

    free = data.get("freeChampionIds", [])
    new = data.get("freeChampionIdsForNewPlayers", [])
    print(f"✅ {len(free)} free champions this week on {platform}")
    print(f"🆕 {len(new)} for new players")
    return data

# Example: check NA1 rotation
rotations = get_champion_rotations("na1")
rotations


✅ 20 free champions this week on na1
🆕 20 for new players


{'freeChampionIds': [5,
  23,
  25,
  29,
  31,
  53,
  55,
  84,
  85,
  106,
  119,
  133,
  154,
  157,
  163,
  360,
  421,
  432,
  516,
  777],
 'freeChampionIdsForNewPlayers': [17,
  18,
  33,
  37,
  51,
  54,
  80,
  82,
  89,
  91,
  112,
  113,
  115,
  131,
  134,
  145,
  222,
  254,
  350,
  875],
 'maxNewPlayerLevel': 10}

In [3]:
def get_champion_list():
    url = "https://ddragon.leagueoflegends.com/cdn/14.22.1/data/en_US/champion.json"
    return requests.get(url).json()["data"]

champions = get_champion_list()
id_to_name = {int(v["key"]): k for k, v in champions.items()}

free_names = [id_to_name.get(cid, cid) for cid in rotations["freeChampionIds"]]
print("Free champions this week:", free_names)


Free champions this week: ['XinZhao', 'Tryndamere', 'Morgana', 'Twitch', 'Chogath', 'Blitzcrank', 'Katarina', 'Akali', 'Kennen', 'Volibear', 'Draven', 'Quinn', 'Zac', 'Yasuo', 'Taliyah', 'Samira', 'RekSai', 'Bard', 'Ornn', 'Yone']


# Summoner V4

In [3]:
import requests, time

API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <-- rotate & paste new key
PUUID   = "0KeRPKJZA9Y6ufAqpft8_N-VpTu6lc1vWrJRpN03_nqXb8znm2Q8vDtpPrltWv8EsEviF4h4LsSOIA"                          # the puuid you have

AMERICAS_PLATFORMS = ["na1", "br1", "la1", "la2", "oc1"]
EUROPE_PLATFORMS   = ["euw1", "eun1", "tr1", "ru"]
ASIA_PLATFORMS     = ["kr", "jp1"]
ALL_PLATFORMS = AMERICAS_PLATFORMS + EUROPE_PLATFORMS + ASIA_PLATFORMS

def _get(url, params=None, tries=3):
    for i in range(tries):
        r = requests.get(url, headers={"X-Riot-Token": API_KEY}, params=params, timeout=20)
        if r.status_code == 200:
            return r.json()
        if r.status_code == 429:
            time.sleep(float(r.headers.get("Retry-After", 1.5*(2**i))))
            continue
        if r.status_code in (401,403):
            raise RuntimeError(f"{r.status_code}: {r.text[:200]} (auth/permissions)")
        if r.status_code == 404:
            return None
        # other codes: brief backoff and retry
        time.sleep(1.5*(2**i))
    raise RuntimeError(f"Failed after {tries} tries")

def summoner_by_puuid(platform, puuid):
    url = f"https://{platform}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
    return _get(url)

# Probe all platforms to find where this PUUID has a Summoner object
found = None
for plat in ALL_PLATFORMS:
    data = summoner_by_puuid(plat, PUUID)
    if data:
        found = (plat, data)
        print(f"✅ Found on {plat}: name={data.get('name')} level={data.get('summonerLevel')}")
        break
    else:
        print(f"… not on {plat}")

if not found:
    raise SystemError("Could not find a Summoner for this PUUID on any platform. "
                      "Double-check the PUUID (is it LoL, not another Riot game?), "
                      "or resolve PUUID via Account-V1 using the player’s Riot ID, then retry.")

# Use the found platform info:
platform, summoner = found
print("Summoner object keys:", list(summoner.keys()))
print("summonerId:", summoner.get("id"))
print("accountId:", summoner.get("accountId"))
print("name:", summoner.get("name"))
print("level:", summoner.get("summonerLevel"))


… not on na1
… not on br1
… not on la1
… not on la2
… not on oc1
… not on euw1
… not on eun1
… not on tr1
… not on ru
✅ Found on kr: name=None level=22
Summoner object keys: ['puuid', 'profileIconId', 'revisionDate', 'summonerLevel']
summonerId: None
accountId: None
name: None
level: 22


# league-v4

In [5]:
# === Cell 0: Setup (auto-detect platforms + queues) ===
import os, time, requests, pandas as pd
from typing import Optional, Dict, Any, List

# --- CONFIG ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"   # your Riot key
PUUID   = None                                           # optional (for later cells)
RETRY_MAX = 5
SLEEP_BETWEEN_REQUESTS = 1.0

# --- Possible platform hosts ---
AMERICAS_PLATFORMS = ["na1","br1","la1","la2","oc1"]
EUROPE_PLATFORMS   = ["euw1","eun1","tr1","ru"]
ASIA_PLATFORMS     = ["kr","jp1"]
ALL_PLATFORMS      = AMERICAS_PLATFORMS + EUROPE_PLATFORMS + ASIA_PLATFORMS

# --- Ranked queues to test ---
ALL_QUEUES = ["RANKED_SOLO_5x5", "RANKED_FLEX_SR", "RANKED_TFT"]

# --- Helpers ---
def _headers() -> Dict[str,str]:
    return {"X-Riot-Token": API_KEY}

def riot_get(url: str, params: Optional[Dict[str,Any]] = None) -> Any:
    last_err = None
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=10)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_REQUESTS)
                return r.json()
            if r.status_code == 429:
                delay = float(r.headers.get("Retry-After", 1.5*(2**attempt)))
                print(f"429 rate-limited; sleeping {delay:.1f}s")
                time.sleep(delay); continue
            if r.status_code in (401,403):
                raise RuntimeError(f"{r.status_code}: auth/permissions error → {r.text[:200]}")
            if r.status_code == 404:
                return None
            last_err = f"{r.status_code}: {r.text[:100]}"
        except requests.RequestException as e:
            last_err = str(e)
        time.sleep(1.0*(2**attempt))
    raise RuntimeError(f"Request failed after retries. Last error: {last_err}")

def entries_to_df(entries: List[dict]) -> pd.DataFrame:
    if not entries: return pd.DataFrame()
    df = pd.DataFrame(entries)
    cols = [c for c in [
        "summonerName","summonerId","leaguePoints","wins","losses",
        "rank","tier","queueType","veteran","inactive","hotStreak","freshBlood","miniSeries"
    ] if c in df.columns]
    return df[cols] if cols else df

# --- Step 1: Platform connectivity check ---
reachable = []
print("🔍 Checking Riot platform hosts...\n")
for plat in ALL_PLATFORMS:
    url = f"https://{plat}.api.riotgames.com/lol/status/v4/platform-data"
    try:
        r = requests.get(url, headers=_headers(), timeout=6)
        if r.status_code == 200:
            reachable.append(plat)
            print(f"✅ {plat} → OK")
        elif r.status_code == 403:
            print(f"❌ {plat} → 403 Forbidden (key expired or not authorized)")
            break
        else:
            print(f"⚠️ {plat} → {r.status_code}")
    except Exception as e:
        print(f"⚠️ {plat} → {e}")

if not reachable:
    raise SystemExit("No reachable platforms — check your API key or network.")
PLATFORM = reachable[0]
print(f"\n➡️ Using first reachable platform: {PLATFORM}")

# --- Step 2: Queue validation on that platform ---
valid_queues = []
print("\n🔍 Checking available queues on", PLATFORM, "...")
for q in ALL_QUEUES:
    url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{q}"
    r = requests.get(url, headers=_headers(), timeout=8)
    if r.status_code == 200:
        valid_queues.append(q)
        print(f"✅ {q} available")
    elif r.status_code == 404:
        print(f"⚠️ {q} not supported on this platform")
    else:
        print(f"⚠️ {q} returned {r.status_code}")

if not valid_queues:
    raise SystemExit("No queues returned successfully — check key or platform access.")
QUEUE = valid_queues[0]
print(f"\n➡️ Using first available queue: {QUEUE}")
print("\nConfig ready ✅")


🔍 Checking Riot platform hosts...

✅ na1 → OK
✅ br1 → OK
✅ la1 → OK
✅ la2 → OK
✅ oc1 → OK
✅ euw1 → OK
✅ eun1 → OK
✅ tr1 → OK
✅ ru → OK
✅ kr → OK
✅ jp1 → OK

➡️ Using first reachable platform: na1

🔍 Checking available queues on na1 ...
✅ RANKED_SOLO_5x5 available
✅ RANKED_FLEX_SR available
⚠️ RANKED_TFT returned 400

➡️ Using first available queue: RANKED_SOLO_5x5

Config ready ✅


In [6]:
# === Cell 1: /lol/league/v4/challengerleagues/by-queue/{queue} ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{QUEUE}"
challenger = riot_get(url)
if not challenger:
    raise SystemExit("No challenger data returned.")

print(f"TIER: {challenger.get('tier')}  |  QUEUE: {challenger.get('queue')}")
chal_df = entries_to_df(challenger.get("entries", []))
print("Challenger entries:", chal_df.shape)
chal_df.head(10)


TIER: CHALLENGER  |  QUEUE: RANKED_SOLO_5x5
Challenger entries: (300, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,2185,509,367,I,True,False,False,False
1,1895,552,458,I,True,False,False,False
2,1782,414,335,I,True,False,False,False
3,1630,347,278,I,True,False,False,False
4,1629,280,178,I,True,False,False,False
5,1597,690,613,I,True,False,False,False
6,1594,313,214,I,True,False,False,False
7,1560,140,76,I,False,False,False,False
8,1522,354,283,I,True,False,False,False
9,1510,287,220,I,True,False,False,False


In [7]:
# === Cell 2: /lol/league/v4/grandmasterleagues/by-queue/{queue} ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{QUEUE}"
grandmaster = riot_get(url)
if not grandmaster:
    raise SystemExit("No grandmaster data returned.")

print(f"TIER: {grandmaster.get('tier')}  |  QUEUE: {grandmaster.get('queue')}")
gm_df = entries_to_df(grandmaster.get("entries", []))
print("Grandmaster entries:", gm_df.shape)
gm_df.head(10)


TIER: GRANDMASTER  |  QUEUE: RANKED_SOLO_5x5
Grandmaster entries: (700, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,911,992,963,I,False,False,True,True
1,852,744,704,I,True,False,True,False
2,852,154,98,I,False,False,True,True
3,850,876,843,I,False,False,False,True
4,850,407,368,I,True,False,False,False
5,848,695,653,I,False,False,False,True
6,838,595,555,I,False,False,False,True
7,835,393,362,I,False,False,False,True
8,832,232,157,I,False,False,False,True
9,829,370,332,I,False,False,False,True


In [8]:
# === Cell 3: /lol/league/v4/masterleagues/by-queue/{queue} ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{QUEUE}"
master = riot_get(url)
if not master:
    raise SystemExit("No master data returned.")

print(f"TIER: {master.get('tier')}  |  QUEUE: {master.get('queue')}")
master_df = entries_to_df(master.get("entries", []))
print("Master entries:", master_df.shape)
master_df.head(10)


TIER: MASTER  |  QUEUE: RANKED_SOLO_5x5
Master entries: (6290, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,548,141,89,I,True,False,True,False
1,522,393,374,I,False,False,True,True
2,519,85,60,I,True,False,False,False
3,518,456,443,I,False,False,True,True
4,515,82,13,I,False,False,True,False
5,508,638,645,I,True,False,True,False
6,505,495,463,I,False,False,False,True
7,505,220,182,I,False,False,True,True
8,504,383,355,I,False,False,True,True
9,503,434,397,I,False,False,True,True


In [9]:
# === Cell 4: /lol/league/v4/entries/{queue}/{tier}/{division}?page=N ===
TIER = "DIAMOND"   # change
DIV  = "I"         # I, II, III, IV

all_pages = []
page = 1
while True:
    url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/entries/{QUEUE}/{TIER}/{DIV}"
    entries = riot_get(url, params={"page": page})
    if not entries:
        break
    all_pages.extend(entries)
    print(f"Loaded page {page}: {len(entries)} rows")
    page += 1
    if page > 5:  # safety cap; remove/raise if you want all pages
        break

entries_df = entries_to_df(all_pages)
print(f"Total {TIER} {DIV} entries:", entries_df.shape)
entries_df.head(10)


Loaded page 1: 205 rows
Loaded page 2: 205 rows
Loaded page 3: 205 rows
Loaded page 4: 205 rows
Loaded page 5: 205 rows
Total DIAMOND I entries: (1025, 10)


,leaguePoints,wins,losses,rank,tier,queueType,veteran,inactive,hotStreak,freshBlood
0,57,263,269,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,True
1,75,40,30,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
2,25,39,54,I,DIAMOND,RANKED_SOLO_5x5,False,False,True,False
3,84,109,93,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
4,75,143,137,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,True
5,56,253,267,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,True
6,97,109,83,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
7,8,61,57,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
8,5,190,184,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False
9,56,249,248,I,DIAMOND,RANKED_SOLO_5x5,False,False,False,False


In [10]:
# === Cell 5: /lol/league/v4/leagues/{leagueId} ===
league_id = (challenger or {}).get("leagueId", None)
if not league_id:
    raise SystemExit("No leagueId found (run Cell 1 first).")

url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/leagues/{league_id}"
league_detail = riot_get(url)
print("League:", league_detail.get("tier"), "|", league_detail.get("queue"))
ld_df = entries_to_df(league_detail.get("entries", []))
print("League detail entries:", ld_df.shape)
ld_df.head(10)


League: CHALLENGER | RANKED_SOLO_5x5
League detail entries: (300, 8)


,leaguePoints,wins,losses,rank,veteran,inactive,hotStreak,freshBlood
0,2185,509,367,I,True,False,False,False
1,1895,552,458,I,True,False,False,False
2,1782,414,335,I,True,False,False,False
3,1630,347,278,I,True,False,False,False
4,1629,280,178,I,True,False,False,False
5,1597,690,613,I,True,False,False,False
6,1594,313,214,I,True,False,False,False
7,1560,140,76,I,False,False,False,False
8,1522,354,283,I,True,False,False,False
9,1510,287,220,I,True,False,False,False


In [11]:
# === Cell 6: /lol/league/v4/entries/by-puuid/{encryptedPUUID} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 to use this call.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/league/v4/entries/by-puuid/{PUUID}"
by_puuid = riot_get(url) or []
print(f"Ranked entries for this player: {len(by_puuid)}")

# Normalize list of dicts → DataFrame
puuid_df = pd.DataFrame(by_puuid)
# reorder common columns if present
cols = [c for c in [
    "queueType","tier","rank","leaguePoints","wins","losses","hotStreak","veteran","inactive","freshBlood"
] if c in puuid_df.columns]
puuid_df = puuid_df[cols] if cols else puuid_df
puuid_df.head(10)


SystemExit: Set PUUID in Cell 0 to use this call.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# === Cell 7: Save JSON & CSV snapshots (optional) ===
import json, os
os.makedirs("output_league_v4", exist_ok=True)

snap = {
    "challenger": challenger,
    "grandmaster": grandmaster,
    "master": master,
    "entries_diamond_I": entries_df.to_dict(orient="records"),
    "league_detail": league_detail if "league_detail" in locals() else None,
    "by_puuid": by_puuid if "by_puuid" in locals() else None,
}
with open("output_league_v4/all_league_v4.json", "w") as f:
    json.dump(snap, f, indent=2)

# Also save a few CSVs
chal_df.to_csv("output_league_v4/challenger_entries.csv", index=False)
gm_df.to_csv("output_league_v4/grandmaster_entries.csv", index=False)
master_df.to_csv("output_league_v4/master_entries.csv", index=False)
entries_df.to_csv("output_league_v4/diamond_I_entries.csv", index=False)
if "puuid_df" in locals():
    puuid_df.to_csv("output_league_v4/by_puuid_entries.csv", index=False)

print("Saved outputs in output_league_v4/")


# League v4 exp

In [13]:
# === League-EXP-V4: entries across ALL platforms & queues (no manual division config) ===
import requests, time, pandas as pd, os

API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <-- put your key here

# Platforms to scan
AMERICAS = ["na1","br1","la1","la2","oc1"]
EUROPE   = ["euw1","eun1","tr1","ru"]
ASIA     = ["kr","jp1"]
ALL_PLATFORMS = AMERICAS + EUROPE + ASIA

# LoL ranked queues (TFT uses a different product; leave it out here)
ALL_QUEUES = ["RANKED_SOLO_5x5", "RANKED_FLEX_SR"]

# Tiers
DIV_TIERS  = ["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
APEX_TIERS = ["MASTER","GRANDMASTER","CHALLENGER"]  # division ignored by Riot for these

# Request controls
RETRY_MAX = 5
BASE_SLEEP = 1.0     # base pacing between calls
OUT_DIR = "output_league_exp_v4_all"

def _get(url, params=None):
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers={"X-Riot-Token": API_KEY}, params=params, timeout=25)
        if r.status_code == 200:
            time.sleep(BASE_SLEEP)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…")
            time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions issue → {r.text[:180]}")
        if r.status_code == 404:  # unsupported combo on that platform/queue
            return None
        last = f"{r.status_code}: {r.text[:180]}"
        time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

def fetch_entries(platform:str, queue:str, tier:str, division:str|None, max_pages:int=50):
    """
    Fetches all pages for a given (platform, queue, tier[, division]).
    For apex tiers, 'division' can be None (we send 'I' only to satisfy the path).
    """
    div_for_path = division if division is not None else "I"   # Riot ignores division for apex, but path requires it
    base = f"https://{platform}.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{div_for_path}"

    rows = []
    page = 1
    while page <= max_pages:
        data = _get(base, params={"page": page})
        if not data:
            break
        # annotate each row
        for d in data:
            d["_platform"] = platform
            d["_queue"]    = queue
            d["_tier"]     = tier
            d["_division"] = division if division is not None else "NA"  # NA = not applicable (apex)
        rows.extend(data)
        # If a page returns < 205 entries, it's usually the last; still try next once
        print(f"{platform} | {queue} | {tier} {div_for_path}: page {page} → {len(data)} rows (total {len(rows)})")
        if len(data) == 0:
            break
        page += 1
        # gentle pacing between pages
        time.sleep(0.3)
    return rows

# -------- Run the sweep ----------
all_rows = []

for plat in ALL_PLATFORMS:
    # quick connectivity probe (optional but helpful)
    probe = requests.get(f"https://{plat}.api.riotgames.com/lol/status/v4/platform-data",
                         headers={"X-Riot-Token": API_KEY}, timeout=8)
    if probe.status_code != 200:
        print(f"Skip {plat} (status={probe.status_code})")
        continue

    for q in ALL_QUEUES:
        # First check if the queue exists on this platform by requesting Challenger apex page 1
        chk = requests.get(
            f"https://{plat}.api.riotgames.com/lol/league-exp/v4/entries/{q}/CHALLENGER/I",
            headers={"X-Riot-Token": API_KEY}, params={"page": 1}, timeout=12
        )
        if chk.status_code not in (200, 404):  # non-fatal but informative
            print(f"  {plat} {q} probe → {chk.status_code}")
        if chk.status_code == 404:
            print(f"  {plat} does not support {q}; skipping.")
            continue

        # 1) Divisional tiers (I..IV)
        for tier in DIV_TIERS:
            for div in ["I","II","III","IV"]:
                try:
                    rows = fetch_entries(plat, q, tier, div, max_pages=50)
                    all_rows.extend(rows)
                except Exception as e:
                    print(f"Error on {plat} {q} {tier} {div}: {e}")

        # 2) Apex tiers (division not applicable; we label 'NA')
        for tier in APEX_TIERS:
            try:
                rows = fetch_entries(plat, q, tier, division=None, max_pages=50)
                all_rows.extend(rows)
            except Exception as e:
                print(f"Error on {plat} {q} {tier}: {e}")

print(f"\nTotal rows collected: {len(all_rows)}")

# -------- DataFrame + save ----------
df = pd.DataFrame(all_rows)
if not df.empty:
    # Clean and order some useful columns if present
    useful = [c for c in [
        "_platform","_queue","_tier","_division",
        "summonerName","summonerId","queueType","tier","rank",
        "leaguePoints","wins","losses","hotStreak","veteran","inactive","freshBlood"
    ] if c in df.columns]
    df = df[useful + [c for c in df.columns if c not in useful]]  # keep extras too

print("DataFrame shape:", df.shape)
display(df.head(20))

os.makedirs(OUT_DIR, exist_ok=True)
csv_path = os.path.join(OUT_DIR, "league_exp_all_platforms_queues.csv")
df.to_csv(csv_path, index=False)
print("Saved →", csv_path)


na1 | RANKED_SOLO_5x5 | IRON I: page 1 → 205 rows (total 205)
na1 | RANKED_SOLO_5x5 | IRON I: page 2 → 205 rows (total 410)
na1 | RANKED_SOLO_5x5 | IRON I: page 3 → 205 rows (total 615)
na1 | RANKED_SOLO_5x5 | IRON I: page 4 → 205 rows (total 820)
na1 | RANKED_SOLO_5x5 | IRON I: page 5 → 205 rows (total 1025)
na1 | RANKED_SOLO_5x5 | IRON I: page 6 → 205 rows (total 1230)
na1 | RANKED_SOLO_5x5 | IRON I: page 7 → 205 rows (total 1435)
na1 | RANKED_SOLO_5x5 | IRON I: page 8 → 205 rows (total 1640)
na1 | RANKED_SOLO_5x5 | IRON I: page 9 → 205 rows (total 1845)
na1 | RANKED_SOLO_5x5 | IRON I: page 10 → 205 rows (total 2050)
na1 | RANKED_SOLO_5x5 | IRON I: page 11 → 205 rows (total 2255)
na1 | RANKED_SOLO_5x5 | IRON I: page 12 → 205 rows (total 2460)
na1 | RANKED_SOLO_5x5 | IRON I: page 13 → 205 rows (total 2665)
na1 | RANKED_SOLO_5x5 | IRON I: page 14 → 205 rows (total 2870)
na1 | RANKED_SOLO_5x5 | IRON I: page 15 → 205 rows (total 3075)
na1 | RANKED_SOLO_5x5 | IRON I: page 16 → 205 rows (t

KeyboardInterrupt: 

this is one of Riot’s data-heavy ranked-ladder APIs. It exposes a paginated listing of every player in a given ranked queue, tier, and division — effectively the “expanded” version of the standard League-V4 /entries call.

# Clash v1

In [14]:
# === Cell 0: Setup (config + platform check + helpers) ===
import os, time, requests, pandas as pd
from typing import Optional, Dict, Any, List

# --- CONFIG (edit these) ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"   # paste your key
PUUID   = None   # set when using Cell 1 (players/by-puuid)
TEAM_ID = None   # set when using Cells 2 & 5
TOURNAMENT_ID = None  # set when using Cell 4

RETRY_MAX = 5
SLEEP_BETWEEN_REQUESTS = 1.0

# Platform list (Clash is platform-routed)
AMERICAS_PLATFORMS = ["na1","br1","la1","la2","oc1"]
EUROPE_PLATFORMS   = ["euw1","eun1","tr1","ru"]
ASIA_PLATFORMS     = ["kr","jp1"]
ALL_PLATFORMS      = AMERICAS_PLATFORMS + EUROPE_PLATFORMS + ASIA_PLATFORMS

def _headers() -> Dict[str, str]:
    return {"X-Riot-Token": API_KEY}

def riot_get(url: str, params: Optional[Dict[str, Any]] = None) -> Any:
    last_err = None
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=20)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_REQUESTS)
                return r.json()
            if r.status_code == 429:
                delay = float(r.headers.get("Retry-After", 1.5 * (2 ** attempt)))
                print(f"429 rate-limited; sleeping {delay:.1f}s")
                time.sleep(delay); continue
            if r.status_code in (401, 403):
                raise RuntimeError(f"{r.status_code}: auth/permissions → {r.text[:200]}")
            if r.status_code == 404:
                return None
            last_err = f"{r.status_code}: {r.text[:160]}"
        except requests.RequestException as e:
            last_err = str(e)
        time.sleep(1.0 * (2 ** attempt))
    raise RuntimeError(f"Request failed after retries. Last error: {last_err}")

# Auto-pick the first reachable platform (status endpoint)
reachable = []
for plat in ALL_PLATFORMS:
    try:
        r = requests.get(f"https://{plat}.api.riotgames.com/lol/status/v4/platform-data",
                         headers=_headers(), timeout=8)
        if r.status_code == 200:
            reachable.append(plat)
            print(f"✅ {plat} OK")
        elif r.status_code == 403:
            print(f"❌ {plat} 403 (key expired/forbidden)"); break
        else:
            print(f"⚠️  {plat} → {r.status_code}")
    except Exception as e:
        print(f"⚠️  {plat} error:", e)

if not reachable:
    raise SystemExit("No reachable platforms — check API key/network.")
PLATFORM = reachable[0]
print(f"\nUsing PLATFORM → {PLATFORM}")


✅ na1 OK
✅ br1 OK
✅ la1 OK
✅ la2 OK
✅ oc1 OK
✅ euw1 OK
✅ eun1 OK
✅ tr1 OK
✅ ru OK
✅ kr OK
✅ jp1 OK

Using PLATFORM → na1


In [15]:
# === Cell 1: /lol/clash/v1/players/by-puuid/{puuid} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 to query players/by-puuid.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/players/by-puuid/{PUUID}"
player_entries = riot_get(url) or []   # list of PlayerDTOs

df_players = pd.DataFrame(player_entries)
print("players/by-puuid rows:", df_players.shape)
df_players.head(10)


SystemExit: Set PUUID in Cell 0 to query players/by-puuid.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
# === Cell 2: /lol/clash/v1/teams/{teamId} ===
if not TEAM_ID:
    raise SystemExit("Set TEAM_ID in Cell 0 to query teams/{teamId}.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/teams/{TEAM_ID}"
team = riot_get(url)
if not team:
    raise SystemExit("No team data returned (404 or empty).")

# Basic fields
team_df = pd.json_normalize({
    "teamId": team.get("id"),
    "tournamentId": team.get("tournamentId"),
    "name": team.get("name"),
    "iconId": team.get("iconId"),
    "tier": team.get("tier"),
    "captain": team.get("captain"),
    "abbreviation": team.get("abbreviation")
})
print("Team info:")
display(team_df)

# Roster (players array)
players_list = team.get("players", []) or []
players_df = pd.json_normalize(players_list)
print("Team players:")
players_df.head(10)


SystemExit: Set TEAM_ID in Cell 0 to query teams/{teamId}.

In [17]:
# === Cell 3: /lol/clash/v1/tournaments ===
url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/tournaments"
tournaments = riot_get(url) or []  # list of TournamentDTO

# Base tournament info
base_cols = ["id","themeId","name"]
base_rows = [{k: t.get(k) for k in base_cols} for t in tournaments]
tournaments_df = pd.DataFrame(base_rows)
print("Tournaments:", tournaments_df.shape)
display(tournaments_df.head(10))

# Explode schedules into a table (one row per schedule entry)
sched_rows = []
for t in tournaments:
    for s in (t.get("schedule") or []):
        row = {"tournamentId": t.get("id")}
        row.update(s)  # usually has "id","registrationTime","startTime","cancelled"
        sched_rows.append(row)
schedules_df = pd.DataFrame(sched_rows)
print("Tournament schedules:", schedules_df.shape)
schedules_df.head(10)


Tournaments: (2, 3)


,id,themeId,name
0,136381,9,None
1,136382,9,None


Tournament schedules: (2, 5)


,tournamentId,id,registrationTime,startTime,cancelled
0,136381,129241,1763856900000,1763866800000,False
1,136382,129242,1763943300000,1763953200000,False


In [18]:
# === Cell 4: /lol/clash/v1/tournaments/{tournamentId} ===
if not TOURNAMENT_ID:
    raise SystemExit("Set TOURNAMENT_ID in Cell 0 to query tournaments/{tournamentId}.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/tournaments/{TOURNAMENT_ID}"
t = riot_get(url)
if not t:
    raise SystemExit("No tournament data returned (404 or empty).")

# Base fields
tournament_df = pd.json_normalize({k: t.get(k) for k in ["id","themeId","name"]})
print("Tournament:")
display(tournament_df)

# Schedule (explode)
sched_df = pd.DataFrame(t.get("schedule") or [])
print("Schedule entries:")
sched_df.head(10)


SystemExit: Set TOURNAMENT_ID in Cell 0 to query tournaments/{tournamentId}.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
# === Cell 5: /lol/clash/v1/tournaments/by-team/{teamId} ===
if not TEAM_ID:
    raise SystemExit("Set TEAM_ID in Cell 0 to query tournaments/by-team/{teamId}.")

url = f"https://{PLATFORM}.api.riotgames.com/lol/clash/v1/tournaments/by-team/{TEAM_ID}"
list_for_team = riot_get(url) or []  # list of TournamentDTOs

team_tourneys_df = pd.DataFrame([{k: t.get(k) for k in ["id","themeId","name"]} for t in list_for_team])
print("Tournaments for team:", team_tourneys_df.shape)
display(team_tourneys_df.head(10))

# All schedules combined
sched_rows = []
for t in list_for_team:
    for s in (t.get("schedule") or []):
        row = {"tournamentId": t.get("id"), "name": t.get("name")}
        row.update(s)
        sched_rows.append(row)
team_sched_df = pd.DataFrame(sched_rows)
print("Schedules for team:", team_sched_df.shape)
team_sched_df.head(10)


SystemExit: Set TEAM_ID in Cell 0 to query tournaments/by-team/{teamId}.

# Account-V1

In [1]:
# === Cell 0: Setup for ACCOUNT-V1 (regional routing) ===
import os, time, requests, pandas as pd
from urllib.parse import quote
from typing import Optional, Dict, Any

# --- CONFIG (edit these) ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"
REGION  = "ASIA"   # "AMERICAS" | "EUROPE" | "ASIA"

# Optional convenience defaults used by cells below
GAME_NAME = "Faker"          # Riot ID name
TAG_LINE  = "KR1"               # Riot ID tagline
PUUID     = None                 # set if you have one (else run Cell 1 first)
GAME_FOR_ACTIVE_SHARD = "lol"    # "lol" | "val" | "lor"

RETRY_MAX = 5
SLEEP_BETWEEN = 0.8

def _acct_base(region: str) -> str:
    bases = {
        "AMERICAS": "https://americas.api.riotgames.com",
        "EUROPE":   "https://europe.api.riotgames.com",
        "ASIA":     "https://asia.api.riotgames.com",
    }
    return bases[region.upper()]

def _headers() -> Dict[str, str]:
    return {"X-Riot-Token": API_KEY}

def acct_get(path: str, params: Optional[Dict[str, Any]] = None, region: Optional[str] = None):
    """GET with light retry/backoff for ACCOUNT-V1 (regional host)."""
    base = _acct_base((region or REGION))
    url = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers=_headers(), params=params, timeout=20)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…")
            time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions → {r.text[:200]}")
        if r.status_code == 404:
            return None
        last = f"{r.status_code}: {r.text[:200]}"
        time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

print("ACCOUNT-V1 ready on regional host:", _acct_base(REGION))


ACCOUNT-V1 ready on regional host: https://asia.api.riotgames.com


In [2]:
# === Cell 1: /riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine} ===
# Returns Account DTO: { puuid, gameName, tagLine }
gn_enc = quote(GAME_NAME, safe="")
tl_enc = quote(TAG_LINE,  safe="")

acct = acct_get(f"/riot/account/v1/accounts/by-riot-id/{gn_enc}/{tl_enc}")
if not acct:
    raise SystemExit("No account found (404). Check name/tag & REGION.")

acct_df = pd.json_normalize(acct)  # single-row DF
print("Resolved Riot ID → PUUID")
acct_df


Resolved Riot ID → PUUID


,puuid,gameName,tagLine
0,0KeRPKJZA9Y6ufAqpft8_N-VpTu6lc1vWrJRpN03_nqXb8...,Faker,KR1


In [3]:
# === Cell 2: /riot/account/v1/accounts/by-puuid/{puuid} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 (or run Cell 1 and copy its puuid).")

acct_by_puuid = acct_get(f"/riot/account/v1/accounts/by-puuid/{PUUID}")
if not acct_by_puuid:
    raise SystemExit("No account for that PUUID (404). Check REGION and PUUID.")

acct_by_puuid_df = pd.json_normalize(acct_by_puuid)
print("Account (by PUUID):")
acct_by_puuid_df


SystemExit: Set PUUID in Cell 0 (or run Cell 1 and copy its puuid).

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# === Cell 3: /riot/account/v1/active-shards/by-game/{game}/by-puuid/{puuid} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 (or resolve in Cell 1).")

active = acct_get(f"/riot/account/v1/active-shards/by-game/{GAME_FOR_ACTIVE_SHARD}/by-puuid/{PUUID}")
if not active:
    raise SystemExit("No active shard returned (404). Player may not have played that game recently.")

active_df = pd.json_normalize(active)  # fields: { puuid, game, activeShard }
print("Active shard for game =", GAME_FOR_ACTIVE_SHARD)
active_df


SystemExit: Set PUUID in Cell 0 (or resolve in Cell 1).

In [5]:
# === Cell 4: Convenience chain (RiotID → PUUID → account/by-puuid → active shard) ===
gn_enc = quote(GAME_NAME, safe="")
tl_enc = quote(TAG_LINE,  safe="")

acct = acct_get(f"/riot/account/v1/accounts/by-riot-id/{gn_enc}/{tl_enc}")
if not acct:
    raise SystemExit("No account for that Riot ID in this REGION.")
puuid = acct["puuid"]

acct2 = acct_get(f"/riot/account/v1/accounts/by-puuid/{puuid}") or {}
active = acct_get(f"/riot/account/v1/active-shards/by-game/{GAME_FOR_ACTIVE_SHARD}/by-puuid/{puuid}") or {}

result_df = pd.DataFrame([{
    "riot_gameName": acct.get("gameName"),
    "riot_tagLine":  acct.get("tagLine"),
    "puuid":         puuid,
    "verify_gameName": acct2.get("gameName"),
    "verify_tagLine":  acct2.get("tagLine"),
    "active_game":     active.get("game"),
    "activeShard":     active.get("activeShard"),
}])
print("RiotID → PUUID → verification + active shard")
result_df


SystemExit: 403 auth/permissions → {"status":{"message":"Forbidden","status_code":403}}

# LOL-Status V4

In [6]:
# === LoL-Status-V4 /lol/status/v4/platform-data (stand-alone) ===
import requests, time, pandas as pd, os

# --- CONFIG ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857" # <- your key
PLATFORMS = ["na1"]  # e.g. ["na1","euw1","kr"]  (see ALL_PLATFORMS below to scan all)

# Uncomment to scan all shards:
# PLATFORMS = ["na1","br1","la1","la2","oc1","euw1","eun1","tr1","ru","kr","jp1"]

RETRY_MAX = 5
SLEEP_BETWEEN = 0.3
OUT_DIR = "output_status_v4"

def _get(url, params=None):
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers={"X-Riot-Token": API_KEY}, params=params, timeout=15)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…"); time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions → {r.text[:180]}")
        if r.status_code == 404:
            return None
        last = f"{r.status_code}: {r.text[:180]}"
        time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

def fetch_platform_status(platform: str):
    url = f"https://{platform}.api.riotgames.com/lol/status/v4/platform-data"
    return _get(url) or {}

# ---- fetch + normalize ----
summary_rows = []
event_rows = []  # incidents + maintenances flattened

for plat in PLATFORMS:
    data = fetch_platform_status(plat)
    if not data:
        print(f"⚠️ No data for {plat}")
        continue

    # Summary
    incidents = data.get("incidents", []) or []
    maints    = data.get("maintenances", []) or []
    locales   = data.get("locales", []) or []

    summary_rows.append({
        "platform": plat,
        "id": data.get("id"),
        "name": data.get("name"),
        "locales_count": len(locales),
        "incidents_count": len(incidents),
        "maintenances_count": len(maints),
        "has_incidents": bool(incidents),
        "has_maintenances": bool(maints),
    })

    # Flatten incidents/maintenances
    def _flatten(items, kind):
        for it in items:
            titles = it.get("titles") or []
            updates = it.get("updates") or []
            title_text = titles[0]["translation"]["content"] if titles and "translation" in titles[0] else None
            # Pull a quick summary from the latest update if present
            latest = updates[0] if updates else {}
            desc = latest.get("translations", [{}])[0].get("content") if latest.get("translations") else None
            event_rows.append({
                "platform": plat,
                "type": kind,  # "incident" or "maintenance"
                "id": it.get("id"),
                "status": it.get("maintenance_status") or it.get("status"),
                "created_at": it.get("created_at"),
                "archive_at": it.get("archive_at"),
                "title": title_text,
                "updates_count": len(updates),
                "short_desc": desc,
            })

    _flatten(incidents, "incident")
    _flatten(maints, "maintenance")

# ---- DataFrames ----
summary_df = pd.DataFrame(summary_rows).sort_values(["has_incidents","has_maintenances","platform"], ascending=[False, False, True])
events_df  = pd.DataFrame(event_rows)

print("Summary shape:", summary_df.shape)
display(summary_df.head(20))
print("\nEvents shape:", events_df.shape)
display(events_df.head(20))

# ---- Save ----
os.makedirs(OUT_DIR, exist_ok=True)
summary_df.to_csv(os.path.join(OUT_DIR, "status_summary.csv"), index=False)
events_df.to_csv(os.path.join(OUT_DIR, "status_events.csv"), index=False)
print(f"Saved → {OUT_DIR}/status_summary.csv and status_events.csv")


Summary shape: (1, 8)


,platform,id,name,locales_count,incidents_count,maintenances_count,has_incidents,has_maintenances
0,na1,NA1,North America,27,1,1,True,True



Events shape: (2, 9)


,platform,type,id,status,created_at,archive_at,title,updates_count,short_desc
0,na1,incident,11938,None,2025-11-04T19:59:54.244398+00:00,None,None,1,We're aware of a problem causing end of game i...
1,na1,maintenance,11921,scheduled,2025-11-03T20:58:53.841743+00:00,2025-11-05T15:00:00+00:00,None,1,"Mulai 16:30 WIB 05/11/2025, Mode Ranked akan d..."


Saved → output_status_v4/status_summary.csv and status_events.csv


# Match-V5

In [8]:
# === Cell 0: MATCH-V5 setup (REGIONAL routing) ===
import os, time, requests, pandas as pd
from typing import Optional, Dict, Any, List

# ---- CONFIG (edit these) ----
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"
REGION  = "AMERICAS"   # "AMERICAS" | "EUROPE" | "ASIA"
PUUID   = None         # set a PUUID to use Cell 1

# Optional filters for Cell 1 (None = no filter)
QUEUE       = None     # e.g., 420 (Ranked Solo), 440 (Flex), etc.
START_TIME  = None     # epoch seconds (inclusive)
END_TIME    = None     # epoch seconds (exclusive)

SLEEP_BETWEEN = 0.8
RETRY_MAX = 5

def _regional_base(region:str) -> str:
    return {
        "AMERICAS": "https://americas.api.riotgames.com",
        "EUROPE":   "https://europe.api.riotgames.com",
        "ASIA":     "https://asia.api.riotgames.com",
    }[region.upper()]

def _headers() -> Dict[str,str]:
    return {"X-Riot-Token": API_KEY}

def match_get(path: str, params: Optional[Dict[str,Any]]=None, region: Optional[str]=None):
    """GET helper for MATCH-V5 with retry/backoff."""
    base = _regional_base(region or REGION)
    url = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers=_headers(), params=params, timeout=25)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…"); time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions → {r.text[:200]}")
        if r.status_code == 404:
            return None
        last = f"{r.status_code}: {r.text[:200]}"; time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")


In [9]:
# === Cell 1: by-puuid → match IDs (paged) ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0.")

ids_all: List[str] = []
start = 0
page_size = 100     # Riot max
max_total = 1000    # safety cap; raise if you want more

while True:
    params = {"start": start, "count": page_size}
    if QUEUE is not None:     params["queue"] = int(QUEUE)
    if START_TIME is not None: params["startTime"] = int(START_TIME)
    if END_TIME   is not None: params["endTime"]   = int(END_TIME)

    ids = match_get(f"/lol/match/v5/matches/by-puuid/{PUUID}/ids", params=params) or []
    if not ids: break
    ids_all.extend(ids)
    print(f"Fetched {len(ids)} (total {len(ids_all)}) [start={start}]")
    start += len(ids)
    if len(ids_all) >= max_total: break

match_ids_df = pd.DataFrame({"matchId": ids_all})
print("Total IDs:", match_ids_df.shape[0])
match_ids_df.head(15)


SystemExit: Set PUUID in Cell 0.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
# === Cell 2: match detail → participants & teams ===
# Provide a list of match IDs to pull:
MATCH_IDS = match_ids_df["matchId"].head(10).tolist() if "match_ids_df" in globals() else []  # or set manually

if not MATCH_IDS:
    raise SystemExit("Set MATCH_IDS (list of matchId strings).")

part_rows, team_rows = [], []
for mid in MATCH_IDS:
    data = match_get(f"/lol/match/v5/matches/{mid}")
    if not data: 
        print(f"⚠️ No data for {mid}"); 
        continue

    info = data.get("info", {})
    teams = info.get("teams", []) or []
    parts = info.get("participants", []) or []

    # participants
    for p in parts:
        row = {
            "matchId": mid,
            "gameDuration": info.get("gameDuration"),
            "queueId": info.get("queueId"),
            "gameVersion": info.get("gameVersion"),
            "gameCreation": info.get("gameCreation"),
            "teamId": p.get("teamId"),
            "summonerName": p.get("summonerName"),
            "championName": p.get("championName"),
            "puuid": p.get("puuid"),
            "kills": p.get("kills"),
            "deaths": p.get("deaths"),
            "assists": p.get("assists"),
            "win": p.get("win"),
            "role": p.get("role"),
            "lane": p.get("lane"),
            "position": p.get("teamPosition"),
            "goldEarned": p.get("goldEarned"),
            "totalDamageDealtToChampions": p.get("totalDamageDealtToChampions"),
            "visionScore": p.get("visionScore"),
            "cs": (p.get("totalMinionsKilled",0) + p.get("neutralMinionsKilled",0)),
        }
        part_rows.append(row)

    # teams
    for t in teams:
        bans = [b.get("championId") for b in (t.get("bans") or [])]
        tr = {
            "matchId": mid,
            "teamId": t.get("teamId"),
            "win": t.get("win"),
            "baronKills": t.get("objectives",{}).get("baron",{}).get("kills"),
            "dragonKills": t.get("objectives",{}).get("dragon",{}).get("kills"),
            "towerKills": t.get("objectives",{}).get("tower",{}).get("kills"),
            "ban_1": bans[0] if len(bans)>0 else None,
            "ban_2": bans[1] if len(bans)>1 else None,
            "ban_3": bans[2] if len(bans)>2 else None,
            "ban_4": bans[3] if len(bans)>3 else None,
            "ban_5": bans[4] if len(bans)>4 else None,
        }
        team_rows.append(tr)

participants_df = pd.DataFrame(part_rows)
teams_df        = pd.DataFrame(team_rows)
print("Participants:", participants_df.shape, "Teams:", teams_df.shape)
display(participants_df.head(10))
display(teams_df.head(10))


SystemExit: Set MATCH_IDS (list of matchId strings).

In [11]:
# === Cell 3: match timeline → events & frames ===
TIMELINE_MATCH_ID = MATCH_IDS[0] if "MATCH_IDS" in globals() and MATCH_IDS else None
if not TIMELINE_MATCH_ID:
    raise SystemExit("Set TIMELINE_MATCH_ID (a single matchId).")

tl = match_get(f"/lol/match/v5/matches/{TIMELINE_MATCH_ID}/timeline")
if not tl:
    raise SystemExit("No timeline for that match.")

info = tl.get("info", {})
frames = info.get("frames", []) or []

# 1) Flatten EVENTS (one row per event)
event_rows = []
for f in frames:
    ts = f.get("timestamp")
    for ev in (f.get("events") or []):
        event_rows.append({
            "matchId": TIMELINE_MATCH_ID,
            "frame_ts": ts,
            "event_ts": ev.get("timestamp"),
            "type": ev.get("type"),
            "participantId": ev.get("participantId"),
            "killerId": ev.get("killerId"),
            "victimId": ev.get("victimId"),
            "assistingParticipantIds": ev.get("assistingParticipantIds"),
            "itemId": ev.get("itemId"),
            "skillSlot": ev.get("skillSlot"),
            "levelUpType": ev.get("levelUpType"),
            "wardType": ev.get("wardType"),
            "monsterType": ev.get("monsterType"),
            "monsterSubType": ev.get("monsterSubType"),
            "towerType": ev.get("towerType"),
            "buildingType": ev.get("buildingType"),
            "laneType": ev.get("laneType"),
            "bounty": ev.get("bounty"),
            "shutdownBounty": ev.get("shutdownBounty"),
            "position_x": (ev.get("position") or {}).get("x"),
            "position_y": (ev.get("position") or {}).get("y"),
        })
events_df = pd.DataFrame(event_rows)

# 2) Flatten PARTICIPANT FRAMES (gold/cs per minute per participant)
pf_rows = []
for f in frames:
    ts = f.get("timestamp")
    pframes = f.get("participantFrames") or {}
    for pid, pf in pframes.items():
        pf_rows.append({
            "matchId": TIMELINE_MATCH_ID,
            "frame_ts": ts,
            "participantId": int(pid),
            "level": pf.get("level"),
            "currentGold": pf.get("currentGold"),
            "totalGold": pf.get("totalGold"),
            "xp": pf.get("xp"),
            "minionsKilled": pf.get("minionsKilled"),
            "jungleMinionsKilled": pf.get("jungleMinionsKilled"),
            "position_x": (pf.get("position") or {}).get("x"),
            "position_y": (pf.get("position") or {}).get("y"),
        })
participant_frames_df = pd.DataFrame(pf_rows)

print("Events:", events_df.shape, "ParticipantFrames:", participant_frames_df.shape)
display(events_df.head(15))
display(participant_frames_df.head(15))


SystemExit: Set TIMELINE_MATCH_ID (a single matchId).

# lol-challenges-v1

In [13]:
# === Cell 0: Setup for LOL-CHALLENGES-V1 (platform routed) ===
import os, time, requests, pandas as pd
from typing import Optional, Dict, Any

# --- CONFIG (edit these) ---
API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"
PLATFORM = "na1"   # e.g., na1, euw1, kr, br1, la1, la2, oc1, eun1, tr1, ru, jp1

# Handy defaults used by specific cells:
CHALLENGE_ID = 303402      # example id; replace with a real one you care about
LEVEL        = "CHALLENGER"  # One of: IRON, BRONZE, SILVER, GOLD, PLATINUM, EMERALD, DIAMOND, MASTER, GRANDMASTER, CHALLENGER
PUUID        = None        # fill if you want to hit /player-data/{puuid}

# retry / pacing
RETRY_MAX = 5
SLEEP_BETWEEN = 0.8

def _headers() -> Dict[str, str]:
    return {"X-Riot-Token": API_KEY}

def plat_get(path: str, params: Optional[Dict[str, Any]] = None, platform: Optional[str] = None):
    """Light retry/backoff GET against a platform host."""
    base = f"https://{(platform or PLATFORM)}.api.riotgames.com"
    url = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers=_headers(), params=params, timeout=25)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…"); time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions → {r.text[:200]}")
        if r.status_code == 404:
            return None
        last = f"{r.status_code}: {r.text[:200]}"
        time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

print("Challenges-V1 ready on:", PLATFORM)


Challenges-V1 ready on: na1


In [14]:
# === Cell 1: /lol/challenges/v1/challenges/percentiles ===
data = plat_get("/lol/challenges/v1/challenges/percentiles") or {}
# Flatten to rows: challengeId, level, percentile
rows = []
for cid, levels in data.items():
    for lvl, pct in (levels or {}).items():
        rows.append({"challengeId": int(cid), "level": lvl, "percentile": pct})
percentiles_all_df = pd.DataFrame(rows).sort_values(["challengeId","level"])
print("Rows:", percentiles_all_df.shape[0])
percentiles_all_df.head(20)


Rows: 3645


,challengeId,level,percentile
7,0,BRONZE,0.296
2,0,CHALLENGER,0.000
4,0,DIAMOND,0.016
1,0,GOLD,0.117
0,0,GRANDMASTER,0.000
8,0,IRON,0.990
6,0,MASTER,0.000
5,0,PLATINUM,0.052
3,0,SILVER,0.216
16,1,BRONZE,0.279


In [15]:
# === Cell 2: /lol/challenges/v1/challenges/{challengeId}/leaderboards/by-level/{level} ===
# Optional page/limit params are supported by Riot; here we expose 'limit'
LIMIT = 200  # up to 200 per request
params = {"limit": LIMIT}

lb = plat_get(f"/lol/challenges/v1/challenges/{CHALLENGE_ID}/leaderboards/by-level/{LEVEL}", params=params) or []
leaderboard_df = pd.DataFrame(lb)
# Common useful columns if present
keep = [c for c in ["position","puuid","value","positionDelta","updateTime"] if c in leaderboard_df.columns]
leaderboard_df = leaderboard_df[keep] if keep else leaderboard_df
print(f"Leaderboard for challenge {CHALLENGE_ID} at {LEVEL}: {leaderboard_df.shape}")
leaderboard_df.head(20)


Leaderboard for challenge 303402 at CHALLENGER: (200, 3)


,position,puuid,value
0,1,9UuFJ6tqz7sSyiF9hUY1cqlLkpzl-JrZlKgbAj8-7PPlE1...,2226
1,2,JrenZtAoasEBVFQMgPC4ULlKndkgSK6g5YDZk7hGCoAYj5...,1768
2,3,UPXnGlxYGXcEr9PdS-s23CyKPqoctbm1e5sBjQuHKNvaE4...,1753
3,4,cb7_MKPY3_hRmxRfpB2_IVftHDuAYhDDcBlDk8vnO0Aoge...,1469
4,5,U2k0s-kn5gQkEvGJYfw0Cz9Qu3BKTFOCzSmhbkXxAxLADs...,1465
5,6,NmouprzSRShZpwFRwQWE1TtTWsnI1lYo61LEZ1Q3wVjRrV...,1434
6,7,A6-C_UrKcCcAFvkB4ywt-LdC-qajxqCzPGZ_YPzjHt_R7U...,1388
7,8,kgRCYuSfvSSgJw6EwubHZnEpVwCpIvi9p3ZY9fsVe6K7Ui...,1336
8,9,d9zfWsevE8ZhOaEpsve8Q0De4AFSIM9aF-knGqjX_GiMrK...,1287
9,10,Pmgzr5V46Iw4FiO7xs2B0GW4c6YHjJ8otj1pLnlKD9jXoT...,1252


In [16]:
# === Cell 3: /lol/challenges/v1/challenges/{challengeId}/percentiles ===
p = plat_get(f"/lol/challenges/v1/challenges/{CHALLENGE_ID}/percentiles") or {}
# p is dict[level → percentile]
one_pct_df = pd.DataFrame([
    {"challengeId": CHALLENGE_ID, "level": lvl, "percentile": pct}
    for lvl, pct in (p or {}).items()
]).sort_values("percentile")
print("Levels for challenge:", one_pct_df.shape[0])
one_pct_df


Levels for challenge: 10


,challengeId,level,percentile
2,303402,BRONZE,0.000
5,303402,SILVER,0.000
6,303402,IRON,0.000
1,303402,CHALLENGER,0.003
3,303402,GRANDMASTER,0.013
9,303402,MASTER,0.050
7,303402,DIAMOND,0.064
0,303402,PLATINUM,0.080
8,303402,GOLD,0.121
4,303402,NONE,1.000


In [17]:
# === Cell 4: /lol/challenges/v1/challenges/{challengeId}/config ===
cfg = plat_get(f"/lol/challenges/v1/challenges/{CHALLENGE_ID}/config")
if not cfg:
    raise SystemExit("No config returned (404?).")

# Normalize key fields
cfg_df = pd.json_normalize({
    "id": cfg.get("id"),
    "state": cfg.get("state"),
    "tags": ",".join(cfg.get("tags") or []),
    "leaderboard": cfg.get("leaderboard"),
    "thresholds_present": bool(cfg.get("thresholds")),
    "translation_name": (cfg.get("localizedNames") or {}).get("en_US", {}).get("name"),
    "translation_shortDesc": (cfg.get("localizedNames") or {}).get("en_US", {}).get("shortDescription"),
})
print("Config for challenge", CHALLENGE_ID)
cfg_df


Config for challenge 303402


,id,state,tags,leaderboard,thresholds_present,translation_name,translation_shortDesc
0,303402,ENABLED,,True,True,"It Has ""Ultimate"" In the Name!",Win with a 5-stack with 3+ <em>champs with a l...


In [18]:
# === Cell 5: /lol/challenges/v1/player-data/{puuid} ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0 to use /player-data/{puuid}.")

pdto = plat_get(f"/lol/challenges/v1/player-data/{PUUID}") or {}
# pdto has: totalPoints, categoryPoints, challenges[], preferences, etc.

# 5a) Total points (single-row)
total_df = pd.json_normalize(pdto.get("totalPoints") or {})
print("Total points:")
display(total_df)

# 5b) Category points table
cat_df = pd.DataFrame(pdto.get("categoryPoints") or [])
print("Category points:")
display(cat_df.head(10))

# 5c) Per-challenge progress (one row per challenge for this player)
chal_df = pd.DataFrame(pdto.get("challenges") or [])
# helpful selection if present
keep = [c for c in [
    "challengeId","level","value","achievedTime","position","percentile","playersInLevel",
    "achieved","capstone","maxValue","thresholds"
] if c in chal_df.columns]
chal_df = chal_df[keep] if keep else chal_df
print("Per-challenge rows:", chal_df.shape)
chal_df.head(20)


SystemExit: Set PUUID in Cell 0 to use /player-data/{puuid}.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
# === Cell 6: /lol/challenges/v1/challenges/config ===
cfgs = plat_get("/lol/challenges/v1/challenges/config") or []
cfgs_df = pd.json_normalize(cfgs)
# Friendly subset if present
subset = [c for c in [
    "id","state","leaderboard","tags","localizedNames.en_US.name","localizedNames.en_US.shortDescription"
] if c in cfgs_df.columns]
cfgs_df = cfgs_df[subset] if subset else cfgs_df
print("Total challenge configs:", cfgs_df.shape)
cfgs_df.head(20)


Total challenge configs: (405, 5)


,id,state,leaderboard,localizedNames.en_US.name,localizedNames.en_US.shortDescription
0,0,ENABLED,True,CRYSTAL,Special rules for Crystal
1,1,ENABLED,False,IMAGINATION,IMAGINATION capstone
2,2,ENABLED,False,EXPERTISE,EXPERTISE capstone
3,3,ENABLED,False,VETERANCY,VETERANCY capstone
4,4,ENABLED,False,TEAMWORK,TEAMWORK capstone
5,5,ENABLED,False,COLLECTION,COLLECTION capstone
6,302100,ENABLED,False,Strategy,Earn progress from challenges in the Strategy ...
7,302101,ENABLED,True,Level 1 Fiesta,Get takedowns before jungle camps spawn
8,302102,ENABLED,True,Farm Champs not Camps,"As jungler, get kills on laners before 10 minutes"
9,302103,ENABLED,True,Overwhelming Presence,"As a laner, in 1 game, get kills outside your ..."


# champion-mastery v4

In [20]:
# === Cell 0: Setup for CHAMPION-MASTERY-V4 (platform-routed) ===
import time, requests, pandas as pd
from typing import Optional, Dict, Any

# --- CONFIG (edit these) ---
API_KEY  = "RGAPI-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
PLATFORM = "na1"      # na1, euw1, kr, br1, la1, la2, oc1, eun1, tr1, ru, jp1
PUUID    = None       # set a player's encrypted PUUID to query
TOP_N    = 10         # used by /top endpoint

RETRY_MAX = 5
SLEEP_BETWEEN = 0.8

def _headers() -> Dict[str, str]:
    return {"X-Riot-Token": API_KEY}

def plat_get(path: str, params: Optional[Dict[str, Any]] = None, platform: Optional[str] = None):
    """GET with light retry/backoff against a platform host."""
    base = f"https://{(platform or PLATFORM)}.api.riotgames.com"
    url = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers=_headers(), params=params, timeout=25)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN)
            return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…"); time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions → {r.text[:200]}")
        if r.status_code == 404:
            return None
        last = f"{r.status_code}: {r.text[:200]}"; time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

print("Champion-Mastery-V4 ready on:", PLATFORM)


Champion-Mastery-V4 ready on: na1


In [21]:
# === Cell 1: all champion masteries for a PUUID ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0.")

masteries = plat_get(f"/lol/champion-mastery/v4/champion-masteries/by-puuid/{PUUID}") or []
# Typical fields per row: championId, championLevel, championPoints, lastPlayTime, chestGranted, tokensEarned
all_masteries_df = pd.DataFrame(masteries).sort_values("championPoints", ascending=False)
print("Rows:", all_masteries_df.shape[0])
all_masteries_df.head(15)


SystemExit: Set PUUID in Cell 0.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [22]:
# === Cell 2: mastery for one champion ===
CHAMPION_ID = 157  # example: Yasuo (numeric championId from Data Dragon)
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0.")

one = plat_get(f"/lol/champion-mastery/v4/champion-masteries/by-puuid/{PUUID}/by-champion/{CHAMPION_ID}")
if not one:
    raise SystemExit("No mastery returned (404?). Check championId or region.")

one_mastery_df = pd.json_normalize(one)  # single-row DF
one_mastery_df


SystemExit: Set PUUID in Cell 0.

In [23]:
# === Cell 3: total mastery score ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0.")

score = plat_get(f"/lol/champion-mastery/v4/scores/by-puuid/{PUUID}")
score_df = pd.DataFrame([{"puuid": PUUID, "masteryScore": score}])
score_df


SystemExit: Set PUUID in Cell 0.

In [24]:
# === Cell 4: top N champion masteries ===
if not PUUID:
    raise SystemExit("Set PUUID in Cell 0.")

top = plat_get(f"/lol/champion-mastery/v4/champion-masteries/by-puuid/{PUUID}/top", params={"count": int(TOP_N)}) or []
top_masteries_df = pd.DataFrame(top).sort_values("championPoints", ascending=False)
print(f"Top {TOP_N} rows:", top_masteries_df.shape[0])
top_masteries_df


SystemExit: Set PUUID in Cell 0.

# tournament-stub-v5

In [25]:
# === Cell 0: Setup for TOURNAMENT-STUB-V5 (REGIONAL routing) ===
import time, requests, pandas as pd
from typing import Optional, Dict, Any, List

API_KEY = "RGAPI-aea7e856-cbe8-4360-8284-089ac6523857"  # <- your key
REGION  = "AMERICAS"   # "AMERICAS" | "EUROPE" | "ASIA"

RETRY_MAX = 5
SLEEP_BETWEEN = 0.8

def _regional_base(region:str) -> str:
    return {
        "AMERICAS": "https://americas.api.riotgames.com",
        "EUROPE":   "https://europe.api.riotgames.com",
        "ASIA":     "https://asia.api.riotgames.com",
    }[region.upper()]

def _headers() -> Dict[str, str]:
    # Tournament endpoints use the same X-Riot-Token header
    return {"X-Riot-Token": API_KEY, "Content-Type": "application/json"}

def tstub_get(path: str, params: Optional[Dict[str, Any]]=None, region: Optional[str]=None):
    base = _regional_base(region or REGION)
    url = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers=_headers(), params=params, timeout=20)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN); return r.json()
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i))); print(f"429; sleep {delay:.1f}s"); time.sleep(delay); continue
        if r.status_code in (401,403): raise SystemExit(f"{r.status_code} auth → {r.text[:200]}")
        if r.status_code == 404: return None
        last = f"{r.status_code}: {r.text[:200]}"; time.sleep(1.0*(2**i))
    raise RuntimeError(f"GET failed after retries. Last: {last}")

def tstub_post(path: str, json: Dict[str, Any], params: Optional[Dict[str, Any]]=None, region: Optional[str]=None):
    base = _regional_base(region or REGION)
    url = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.post(url, headers=_headers(), json=json, params=params, timeout=20)
        if r.status_code in (200, 201):  # stub returns 200 with id or list
            time.sleep(SLEEP_BETWEEN); 
            try: 
                return r.json()
            except ValueError:
                # some POSTs just return a plain number or string; handle text
                return r.text
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i))); print(f"429; sleep {delay:.1f}s"); time.sleep(delay); continue
        if r.status_code in (401,403): raise SystemExit(f"{r.status_code} auth → {r.text[:200]}")
        last = f"{r.status_code}: {r.text[:200]}"; time.sleep(1.0*(2**i))
    raise RuntimeError(f"POST failed after retries. Last: {last}")

print("Tournament STUB ready on", _regional_base(REGION))


Tournament STUB ready on https://americas.api.riotgames.com


In [26]:
# === Cell 1: Create provider ===
# 'region' must be a platform tag (na, euw, kr, etc.) in UPPERCASE for the game routing.
# 'url' is your callback webhook for real Tournament API; in STUB it's not called, but must be valid-looking.
provider_payload = {
    "region": "NA",
    "url": "https://example.com/callback"
}
provider_id = tstub_post("/lol/tournament-stub/v5/providers", json=provider_payload)
print("providerId:", provider_id)


providerId: 1


In [27]:
# === Cell 2: Create tournament ===
if not str(provider_id).strip():
    raise SystemExit("Run Cell 1 first to get providerId.")

tournament_payload = {
    "name": "My Test Tournament",
    "providerId": int(provider_id),
}
tournament_id = tstub_post("/lol/tournament-stub/v5/tournaments", json=tournament_payload)
print("tournamentId:", tournament_id)


tournamentId: 2


In [31]:
# === FIXED Cell: Create tournament codes (Stub V5 uses QUERY PARAMS for tournamentId & count) ===

# 1) Ensure provider_id
try:
    _ = int(provider_id)
except Exception:
    print("ℹ️ Creating provider…")
    provider_payload = {"region": "NA", "url": "https://example.com/callback"}
    provider_id = tstub_post("/lol/tournament-stub/v5/providers", json=provider_payload)
    print("✅ providerId:", provider_id)

# 2) Ensure tournament_id
try:
    _ = int(tournament_id)
except Exception:
    print("ℹ️ Creating tournament…")
    tournament_payload = {"name": "My Test Tournament", "providerId": int(provider_id)}
    tournament_id = tstub_post("/lol/tournament-stub/v5/tournaments", json=tournament_payload)
    print("✅ tournamentId:", tournament_id)

# 3) Create codes
#    IMPORTANT: tournamentId & count go in params, NOT in JSON body.
params = {"tournamentId": int(tournament_id), "count": 3}
lobby_config = {
    "mapType": "SUMMONERS_RIFT",
    "pickType": "TOURNAMENT_DRAFT",
    "spectatorType": "ALL",
    "teamSize": 5,
    "metadata": "test-metadata",
    "allowedSummonerIds": []  # optional
}

codes = tstub_post("/lol/tournament-stub/v5/codes", json=lobby_config, params=params)
codes = codes if isinstance(codes, list) else [codes]

codes_df = pd.DataFrame({"tournamentCode": codes})
print(f"✅ Created {len(codes)} code(s) for tournament {tournament_id}")
codes_df


✅ Created 3 code(s) for tournament 2


,tournamentCode
0,STUB04fac-f8bf81ef-b4ef-46aa-aa19-14ed413b94de
1,STUB04fac-541e6acb-9ba1-4522-a3fd-04d64dba2875
2,STUB04fac-79ff2b8c-a053-4b78-bc17-ed7a53bb2fff


In [32]:
# === Cell 4: Get code settings ===
if 'codes_df' not in globals() or codes_df.empty:
    raise SystemExit("Run Cell 3 to create codes first.")
code = codes_df["tournamentCode"].iloc[0]

code_info = tstub_get(f"/lol/tournament-stub/v5/codes/{code}") or {}
code_info_df = pd.json_normalize(code_info)
print("Settings for code:", code)
code_info_df


Settings for code: STUB04fac-f8bf81ef-b4ef-46aa-aa19-14ed413b94de


,id,providerId,tournamentId,code,region,map,teamSize,pickType,lobbyName,password,metaData
0,0,1,2,STUB04fac-15a60815-f73a-47f4-b96d-b63b17f71beb,STUB,SUMMONERS_RIFT,5,DRAFT_MODE,STUB_LOBBY,1ff76bfe6d8ea441ff33336443d307,metadata


In [33]:
# === Cell 5: Get lobby events for a code ===
if 'codes_df' not in globals() or codes_df.empty:
    raise SystemExit("Run Cell 3 to create codes first.")
code = codes_df["tournamentCode"].iloc[0]

events = tstub_get(f"/lol/tournament-stub/v5/lobby-events/by-code/{code}") or {}
# shape typically: {"eventList": [ { "eventType": "...", "summonerId": "...", "timestamp": "..." }, ... ] }
events_df = pd.DataFrame(events.get("eventList") or [])
print("Events rows:", events_df.shape[0], "for code:", code)
events_df.head(20)


Events rows: 3 for code: STUB04fac-f8bf81ef-b4ef-46aa-aa19-14ed413b94de


,timestamp,eventType
0,Tue Nov 04 20:49:03 UTC 2025,PracticeGameCreatedEvent
1,Tue Nov 04 20:49:03 UTC 2025,ChampSelectStartedEvent
2,Tue Nov 04 20:49:03 UTC 2025,GameAllocationStartedEvent


# spectator-v5

In [34]:
# === Cell 0: Setup for SPECTATOR-V5 (platform routed) ===
import time, requests, pandas as pd
from typing import Optional, Dict, Any

# --- CONFIG (edit) ---
API_KEY  = "RGAPI-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
PLATFORM = "na1"                   # na1, euw1, kr, br1, la1, la2, oc1, eun1, tr1, ru, jp1
PUUID    = None                    # encrypted PUUID for the player to check
FALLBACK_ENCRYPTED_SUMMONER_ID = None  # optional: if your shard expects encryptedSummonerId instead

RETRY_MAX = 5
SLEEP_BETWEEN = 0.7

def _headers() -> Dict[str, str]:
    return {"X-Riot-Token": API_KEY}

def _plat_base(platform: str) -> str:
    return f"https://{platform}.api.riotgames.com"

def spec_get(path: str, params: Optional[Dict[str, Any]] = None, platform: Optional[str] = None):
    base = _plat_base(platform or PLATFORM)
    url  = f"{base}{path}"
    last = None
    for i in range(RETRY_MAX):
        r = requests.get(url, headers=_headers(), params=params, timeout=20)
        if r.status_code == 200:
            time.sleep(SLEEP_BETWEEN); return r.json()
        if r.status_code == 404:
            return None  # not currently in game (or id not found on this platform)
        if r.status_code == 429:
            delay = float(r.headers.get("Retry-After", 1.5*(2**i)))
            print(f"429 rate-limited; sleeping {delay:.1f}s…"); time.sleep(delay); continue
        if r.status_code in (401,403):
            raise SystemExit(f"{r.status_code} auth/permissions → {r.text[:200]}")
        last = f"{r.status_code}: {r.text[:200]}"; time.sleep(1.0*(2**i))
    raise RuntimeError(f"Request failed after retries. Last: {last}")

print("Spectator-V5 ready on:", PLATFORM)


Spectator-V5 ready on: na1


In [35]:
# === Cell 1: /lol/spectator/v5/active-games/by-summoner/{encryptedPUUID} ===
if not PUUID and not FALLBACK_ENCRYPTED_SUMMONER_ID:
    raise SystemExit("Set PUUID (preferred) or FALLBACK_ENCRYPTED_SUMMONER_ID in Cell 0.")

# Try by PUUID first (as per your spec image). If no data, optionally fall back to encryptedSummonerId.
resp = None
if PUUID:
    resp = spec_get(f"/lol/spectator/v5/active-games/by-summoner/{PUUID}")

if resp is None and FALLBACK_ENCRYPTED_SUMMONER_ID:
    print("ℹ️ No match via PUUID; trying encryptedSummonerId fallback…")
    resp = spec_get(f"/lol/spectator/v5/active-games/by-summoner/{FALLBACK_ENCRYPTED_SUMMONER_ID}")

if resp is None:
    print("Player is not currently in an active game (or wrong platform/id).")
else:
    # --- Summary row ---
    summary_cols = [
        "gameId","gameMode","gameType","gameQueueConfigId",
        "mapId","gameStartTime","gameLength","platformId"
    ]
    summary = {k: resp.get(k) for k in summary_cols}
    summary_df = pd.DataFrame([summary])

    # --- Participants (10 rows) ---
    parts = resp.get("participants", []) or []
    participants_df = pd.json_normalize(parts)
    # common friendly subset if present:
    keep = [c for c in [
        "summonerName","puuid","teamId","championId","spell1Id","spell2Id",
        "bot","profileIconId","gameCustomizationObjects","perks.perkIds"
    ] if c in participants_df.columns]
    if keep: participants_df = participants_df[keep]

    # --- Banned champions ---
    bans = resp.get("bannedChampions", []) or []
    bans_df = pd.DataFrame(bans)  # columns: championId, teamId, pickTurn (if present)

    # --- Observers (encryptionKey) ---
    observers_df = pd.json_normalize(resp.get("observers") or {})

    print("Summary:", summary_df.shape, "| Participants:", participants_df.shape, "| Bans:", bans_df.shape)
    display(summary_df)
    display(participants_df)
    display(bans_df)
    if not observers_df.empty:
        print("Observers info:")
        display(observers_df)


SystemExit: Set PUUID (preferred) or FALLBACK_ENCRYPTED_SUMMONER_ID in Cell 0.

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
